In [ ]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torchvision
from torchvision import transforms
from torchvision.transforms import ToTensor,Resize, Grayscale, Normalize
import matplotlib.pyplot as plt
import zipfile
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
import pandas as pd
from torch.utils.data import Dataset
import glob
import cv2
import torch
from tqdm import tqdm
import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter

In [ ]:
!unzip train_x1.zip

In [ ]:
class My_class(Dataset):
    def __init__(self, classes_paths, transforms=None, target_transform=None):
        #print(glob.glob(data_dir))
        self.classes = []
        self.images = []
        self.transform = transforms
        for actual_class in classes_paths:
            if actual_class == 1:
                print("")
            paths = classes_paths[actual_class]
            for path in paths:
                # print("Loading " + path)
                for top, dirs, files in os.walk(path):
                    for i, name in enumerate(files):
                        if not os.path.isfile(top + "//" + name):
                            continue
                        try:
                            imag = cv2.imread(top + "//" + name)
                        except Exception:
                            continue
                        self.images.append(imag)
                        self.classes.append(actual_class)
        self.images, self.classes = np.array(self.images), torch.tensor(np.array(self.classes), dtype=torch.int64)


    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        if self.transform==None:
            return self.images[idx], self.classes[idx]
        else:
            return self.transform(np.array(self.images[idx]).astype(np.float32)), self.classes[idx]

In [ ]:
CLASSES_PATHS = {
    0: ["train_x1//SMD0402_CL//correct"],
    1: ["train_x1//SMD0402_R//correct"],
    2: ["train_x1//SMD0603_CL//correct"],
    3: ["train_x1//SMD0603_D//correct"],
    4: ["train_x1//SMD0603_R//correct"],
    5: ["train_x1//SMD0805_CL//correct"],
    6: ["train_x1//SMD0805_R//correct"],
    7: ["train_x1//SMD1206_C//correct"],
    8: ["train_x1//SMD1206_R//correct"],
    9: ["train_x1//SMD1210_C//correct"],
    10: ["train_x1//2-SMD//correct"],
    11: ["train_x1//SMA//correct"],
    12: ["train_x1//SMB//correct"],
    13: ["train_x1//SOD110//correct"],
    14: ["train_x1//SOD323F//correct"],
    15: ["train_x1//SOD523//correct"],
    16: ["train_x1//SOT23-5//correct"],
    17: ["train_x1//SOT23-6//correct"],
    18: ["train_x1//SOT143//correct"],
    19: ["train_x1//SOT323//correct"],
    20: ["train_x1//SOT323-5//correct"],
    21: ["train_x1//SOT343//correct"],
    22: ["train_x1//SOT363//correct"],
    23: ["train_x1//SOT523//correct"],
    24: ["train_x1//SOT723//correct"],
    25: ["train_x1//DIP-%d//correct"],
    26: ["train_x1//LQFP0.4-%d//correct"],
    27: ["train_x1//LQFP0.5-%d&TFSOP-%d//correct"],
    28: ["train_x1//LQFP0.8-%d//correct"],
    29: ["train_x1//LQFP0.65-%d&SSOP-%d//correct"],
    30: ["train_x1//SOIC-%d//correct"]
}
transforms = torchvision.transforms.Compose([ToTensor(),
                                             Grayscale(1),
                                             Resize([32, 32]),
                                             Normalize(mean=0, std=1)])

dataset = My_class(CLASSES_PATHS, transforms)

In [ ]:
images, labels = [], []
for img, label in dataset:
    images.append(img)
    labels.append(label)

In [ ]:
images = torch.stack(images)
labels = torch.stack(labels)

In [ ]:
images.squeeze().shape

In [ ]:
dimages = images.view(8164, 1024)
dimages.shape

In [ ]:
import tensorflow as tf
import tensorboard as tb
tf.io.gfile = tb.compat.tensorflow_stub.io.gfile

import os
import torch
import os
import shutil

# launching Tensorboard in Colab
def reinit_tensorboard(clear_log = True):
  # Log files are read from this directory: 
  logs_base_dir = "runs"
  if clear_log:
    # clear logs 
    #!rm -rfv {logs_base_dir}/*
    shutil.rmtree(logs_base_dir, ignore_errors = True)
    os.makedirs(logs_base_dir, exist_ok=True)
  # Colab magic
  %load_ext tensorboard
  %tensorboard --logdir {logs_base_dir}

In [ ]:
from torch.utils.tensorboard import SummaryWriter
import tensorflow as tf
import tensorboard as tb
import numpy
tf.io.gfile = tb.compat.tensorflow_stub.io.gfile

reinit_tensorboard()

In [ ]:
writer = SummaryWriter(comment = "EyePoint")
np_f = numpy.array(dimages)
writer.add_embedding(mat=np_f, metadata=labels)
writer.close()